### Dataset available [here](https://www.sciencedirect.com/science/article/pii/S2352340918315191?via%3Dihub)

# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# load data
data_h1 = pd.read_csv('./dataset/H1.csv')
data_h2 = pd.read_csv('./dataset/H2.csv')
data_h1['Hotel'] = 'Resort Hotel'
data_h2['Hotel'] = 'City Hotel'
data = pd.concat([data_h1, data_h2], ignore_index=True)

data.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,Hotel
0,0,342,2015,July,27,1,0,0,2,0.0,...,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,Resort Hotel
1,0,737,2015,July,27,1,0,0,2,0.0,...,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01,Resort Hotel
2,0,7,2015,July,27,1,0,1,1,0.0,...,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,Resort Hotel
3,0,13,2015,July,27,1,0,1,1,0.0,...,304,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02,Resort Hotel
4,0,14,2015,July,27,1,0,2,2,0.0,...,240,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03,Resort Hotel


# Data Cleaning

In [2]:
# Remove duplicates
data_cleaned = data.drop_duplicates()

# Data Splitting

Instead of making a stratified split by the outcome (IsCanceled) based on the dimension of the booking creation date, an approach usually employed in time series is applied, convenience splitting. Order the dataset by arrival date of bookings and create blocks of "month/year". Merge a 75% stratified split of each block into a training dataset and the remaining 25% into a test dataset.

In [3]:
from sklearn.model_selection import train_test_split, TimeSeriesSplit

def split_data(data):
    """
    Perform stratified split on the data

    Parameters
    ----------
    data : pd.DataFrame
        The data to split

    Returns
    -------
    X_train : pd.DataFrame
        The training data
    X_test : pd.DataFrame   
        The testing data
    y_train : pd.Series
        The training target
    y_test : pd.Series
        The testing target
    """
    
    data = data.sort_values(by=['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth'])
    
    X = data.drop(columns=['IsCanceled'])
    y = data['IsCanceled']
    
    X['YearMonth'] = X['ArrivalDateYear'].astype(str) + '-' + X['ArrivalDateMonth'].astype(str)
    
    X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []
    
    # Loop over each block and perform stratified split
    for block in X['YearMonth'].unique():
        X_block = X[X['YearMonth'] == block].drop(columns=['YearMonth'])
        y_block = y[X['YearMonth'] == block]
        
        X_train_block, X_test_block, y_train_block, y_test_block = train_test_split(
            X_block, y_block, test_size=0.25, stratify=y_block, random_state=42)
        
        X_train_list.append(X_train_block)
        X_test_list.append(X_test_block)
        y_train_list.append(y_train_block)
        y_test_list.append(y_test_block)
    
    X_train = pd.concat(X_train_list)
    X_test = pd.concat(X_test_list)
    y_train = pd.concat(y_train_list)
    y_test = pd.concat(y_test_list)

    return X_train, X_test, y_train, y_test

def split_data_time_series(data, target_column, n_splits=5):
    """
    Perform time series split on the data

    Parameters
    ----------
    data : pd.DataFrame
        The data to split
    target_column : str
        The target column
    n_splits : int
        The number of splits to perform

    Returns
    -------
    X_train : pd.DataFrame
        The training data
    X_test : pd.DataFrame
        The testing data
    y_train : pd.Series
        The training target
    y_test : pd.Series  
        The testing target
    """
    data = data.sort_values(by=['ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth'])
    
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Ensure stratification by checking the distribution of the target variable
        if np.all(np.bincount(y_train) > 0) and np.all(np.bincount(y_test) > 0):
            break
    
    return X_train, X_test, y_train, y_test


# Feature Selection and Engineering

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# Custom transformer for feature engineering
class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features, cardinality_threshold=0.02):
        self.categorical_features = categorical_features
        self.cardinality_threshold = cardinality_threshold
        self.encoders = {}
        self.imputers = {}
        pass
    
    def fit(self, X, y=None):
        self.fit_categorical_encodings(X, y)
        return self
    
    def transform(self, X):
        X = X.copy()

        # Replace LeadTime with LiveTime
        X['LiveTime'] = X.apply(self.calculate_livetime, axis=1)
        
        # Replace ADR with ADRThirdQuartileDeviation
        X = self.calculate_adr_third_quartile_deviation(X)
        
        # Apply categorical encodings
        X = self.apply_categorical_encodings(X)
        
        # Drop the original columns
        columns_to_drop = ['LeadTime', 'ADR', 'ArrivalDateMonth', 'ArrivalDateDayOfMonth', 
                           'ArrivalDateWeekNumber', 'ArrivalDateYear', 'Country', "AssignedRoomType",
                           'RequiredCarParkingSpaces', 'ReservedRoomType']
        columns_to_drop.extend(self.categorical_features)
        X.drop(columns=columns_to_drop, inplace=True)
        
        return X
    
    def calculate_livetime(self, row):
        arrival_date = datetime.date(row['ArrivalDateYear'], row['ArrivalDateMonth'], row['ArrivalDateDayOfMonth'])
        booking_date = arrival_date - datetime.timedelta(days=row['LeadTime'])
        reservation_status_date = datetime.datetime.strptime(row['ReservationStatusDate'], '%Y-%m-%d').date()

        if row['ReservationStatus'] == 'Check-Out' or row['ReservationStatus'] == 'No-Show':
            return row['LeadTime']
        else:
            # If the reservation is canceled, the live time is the difference between the booking date and the cancel date
            return (reservation_status_date - booking_date).days
    
    def calculate_adr_third_quartile_deviation(self, X):
        # Group by the required factors
        grouped = X.groupby(['DistributionChannel', 'ReservedRoomType', 'ArrivalDateWeekNumber', 'ArrivalDateYear'])

        # Calculate the third quartile for each group
        adr_third_quartile = grouped['ADR'].transform(lambda x: x.quantile(0.75))

        # Calculate the deviation
        X['ADRThirdQuartileDeviation'] = X['ADR'] / adr_third_quartile

        # Handle cases where third quartile is 0 (unlikely, but possible)
        X['ADRThirdQuartileDeviation'] = X['ADRThirdQuartileDeviation'].fillna(0)

        return X
    
    def fit_categorical_encodings(self, X, y):
        for feature in self.categorical_features:
            # Calculate value counts and keep only those above the threshold
            value_counts = X[feature].value_counts(normalize=True)
            frequent_categories = value_counts[value_counts >= self.cardinality_threshold].index.tolist()
            
            # Fit OneHotEncoder for frequent categories
            onehot = OneHotEncoder(handle_unknown='ignore')
            onehot.fit(X[feature].values.reshape(-1, 1))
            self.encoders[feature] = onehot
            
            # Calculate logit-odds
            feature_grouped = pd.DataFrame({
                'total': X.groupby(feature).size(),
                'positive': X.groupby(feature)[y.name].sum()
            })
            feature_grouped['negative'] = feature_grouped['total'] - feature_grouped['positive']
            feature_grouped['logit_odds'] = np.log((feature_grouped['positive'] + 0.5) / (feature_grouped['negative'] + 0.5))
            
            # Fit imputer for logit-odds
            imputer = SimpleImputer(strategy='median')
            imputer.fit(feature_grouped['logit_odds'].values.reshape(-1, 1))
            self.imputers[feature] = imputer

    def apply_categorical_encodings(self, X):
        for feature in self.categorical_features:
            # Apply OneHotEncoder
            onehot_encoded = self.encoders[feature].transform(X[feature].values.reshape(-1, 1))
            onehot_columns = [f"{feature}_onehot_{i}" for i in range(onehot_encoded.shape[1])]
            X[onehot_columns] = onehot_encoded
            
            # Apply logit-odds encoding
            logit_odds = self.imputers[feature].transform(X[feature].map(self.logit_odds_map[feature]).values.reshape(-1, 1))
            X[f"{feature}_logit_odds"] = logit_odds
            
            # Calculate prevalence
            X[f"{feature}_prevalence"] = X[feature].map(self.value_counts[feature])
        
        return X

In [12]:

def preprocess_data(data, categorical_features):
    
    # 1. Perform time-series stratified split
    X_train, X_test, y_train, y_test = split_data(data)
    # X_train, X_test, y_train, y_test = split_data_time_series(data, 'IsCanceled', n_splits=5)
    
    # 2. Feature Engineering
    feature_engineer = FeatureEngineering(categorical_features)
    feature_engineer.fit(X_train, y_train)
    X_train_transformed = feature_engineer.transform(X_train)
    X_test_transformed = feature_engineer.transform(X_test)
    
    return X_train_transformed, X_test_transformed, y_train, y_test


In [13]:
categorical_features = ['Hotel', 
                        'Meal', 
                        'MarketSegment', 
                        'DistributionChannel', 
                        'DepositType', 
                        'CustomerType', 
                        'ReservationStatus',
                        'ReservationStatusDate',
                        'Agent',
                        'Company']

X_train, X_test, y_train, y_test = preprocess_data(data_cleaned, categorical_features)

KeyError: 'Column not found: IsCanceled'

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

def create_model_pipeline(classifier):
    return Pipeline([
        ('feature_engineering', FeatureEngineering(categorical_features)),
        ('classifier', classifier)
    ])

def evaluate_model(X, y, model_pipeline, cv=5):
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score),
        'f1': make_scorer(f1_score)
    }
    
    results = cross_validate(model_pipeline, X, y, cv=skf, scoring=scoring)
    
    return {metric: np.mean(scores) for metric, scores in results.items() if metric.startswith('test_')}

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_pipeline = create_model_pipeline(RandomForestClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from sklearn.linear_model import LogisticRegression

model_pipeline = create_model_pipeline(LogisticRegression())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_pipeline = create_model_pipeline(DecisionTreeClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from xgboost import XGBClassifier

model_pipeline = create_model_pipeline(XGBClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_pipeline = create_model_pipeline(GradientBoostingClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model_pipeline = create_model_pipeline(AdaBoostClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from sklearn.ensemble import BaggingClassifier

model_pipeline = create_model_pipeline(BaggingClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)

In [ ]:
from sklearn.ensemble import BayesianClassifier

model_pipeline = create_model_pipeline(BayesianClassifier())
results = evaluate_model(data_cleaned.drop('IsCanceled', axis=1), data_cleaned['IsCanceled'], model_pipeline)
print(results)